In [15]:

import os 
import pandas as pd
import numpy as np
# git 
os.chdir("/home/ptruong/git/dia_sum/scripts/PXD002952")
from triqler_output_to_df import parse_triqler
from q_value import qvalues


# db
from get_protein_specie_map_from_fasta import fasta_to_protein_specie_map
os.chdir("/home/ptruong/git/dia_sum/database")
protein_specie_map = fasta_to_protein_specie_map("2021-04-27-decoys-reviewed-contam-UP000005640-UP000002311-UP000000625.fas")
#protein_specie_map = protein_specie_map.set_index("protein").T.to_dict()
# MSFragger 
os.chdir("/hdd_14T/data/PXD002952/res_20210530_DIAUmpire/MSFragger")


In [16]:
df = pd.read_csv("diaNN.tsv", sep = "\t")
triq = pd.read_csv("triqler_input_diann.csv", sep = "\t")                      
def compute_triqler_top3_submodule(run):
    triq_run = triq[triq.run == run]                    
    def triqler_top3(triq_run):
        res = triq_run.groupby("proteins")["intensity"].apply(lambda x: x.nlargest(3).mean() if len(x.nlargest(3)) >= 2 else np.nan).reset_index()
        return res
    
    def triqler_printout_unique_peptides_proteins(run):
        triq_run = triq[triq.run == run]
        condition = triq_run.condition.unique()
        print(f"run : {run} - condition : {condition}")
        print(f"Unique peptides detected: {len(triq_run.peptide.unique())}")
        print(f"Unique proteins detected: {len(triq_run.proteins.unique())}")
        print()
    
    triqler_printout_unique_peptides_proteins((run))
    
    res = triqler_top3(triq_run)
    def remove_decoy_tag_protein(protein):
        if protein.split("_")[0] == "DECOY":
            return protein.split("_")[1]
        else: return protein.split("_")[0]
    
    res["proteins_nonTagged"] = res.proteins.map(remove_decoy_tag_protein)
    res["specie"] = res.proteins_nonTagged.map(protein_specie_map)
    experiment_id = triq_run.run.unique()[0]
    sample_id = triq_run.condition.unique()[0]
    res["ProteinName"] = res["proteins"]
    midx = pd.MultiIndex(levels = [[sample_id],[experiment_id]], codes = [[0],[0]], names = ["sample_id", "experiment_id"])
    res = res.set_index(["specie", "ProteinName"])
    res = res.drop(["proteins", "proteins_nonTagged"], axis = 1)
    df = pd.DataFrame(res.values, columns = midx, index = res.index)
    return df

In [3]:
run =  triq.run.unique()[0]

In [4]:
df = compute_triqler_top3_submodule(run)

run : 002-Pedro - condition : [1]
Unique peptides detected: 41895
Unique proteins detected: 6629



In [6]:
    triq_run = triq[triq.run == run]                    

In [10]:
triq_run[triq_run.searchScore > 0.99]

,run,condition,charge,searchScore,intensity,peptide,proteins
0,002-Pedro,1,2,0.999996,765.868,AAAAAAAAAAGAAGGR,Q86U42
6,002-Pedro,1,3,0.999921,236.895,AAAAAAAGDSDSWDADAFSVEDPVRK,O75822
12,002-Pedro,1,2,1.000000,590.563,AAAAAAGAASGLPGPVAQGLK,Q96P70
18,002-Pedro,1,3,0.999999,312.074,AAAAAAGAASGLPGPVAQGLK,Q96P70
24,002-Pedro,1,2,0.999985,272.851,AAAAAAGAGPEMVR,P28482
...,...,...,...,...,...,...,...
337272,002-Pedro,1,2,0.999764,218.762,YYVLNALK,Q6P2Q9
337282,002-Pedro,1,2,1.000000,4610.740,YYVTIIDAPGHR,P68104
337288,002-Pedro,1,3,1.000000,6668.890,YYVTIIDAPGHR,P68104
337297,002-Pedro,1,1,0.997973,37.384,YYYIPQYK,Q8N183


In [17]:
df = pd.read_csv("diaNN.tsv", sep = "\t")


In [18]:
df

,File.Name,Run,Protein.Group,Protein.Ids,Protein.Names,Genes,PG.Quantity,PG.Normalised,PG.MaxLFQ,Genes.Quantity,...,Decoy.Evidence,Decoy.CScore,Fragment.Quant.Raw,Fragment.Quant.Corrected,Fragment.Correlations,MS2.Scan,IM,iIM,Predicted.IM,Predicted.iIM
0,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_002-Pedr...,Q86U42,Q86U42,NaN,PABPN1,883.332,940.000,945.299,883.332,...,1.32546,0.032706,273.041;275.3;282.313;250.384;0;245.065;,273.041;265.163;282.313;250.384;0;242.443;,0.992804;0.98272;0.986664;0.991373;0;0.986925;,38696,0,-1,0,0
1,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_003-Pedr...,Q86U42,Q86U42,NaN,PABPN1,1081.830,1089.970,1056.590,1081.830,...,1.13788,0.004172,303.646;261.01;274.781;250.913;420.347;247.49;,303.646;259.857;274.781;250.913;420.347;244.261;,0.98762;0.996155;0.982998;0.99413;0.959578;0.9...,38760,0,-1,0,0
2,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_004-Pedr...,Q86U42,Q86U42,NaN,PABPN1,1023.520,978.061,989.465,1023.520,...,1.26365,0.004910,333.183;310.885;321.896;266.452;0;274.825;,333.183;310.885;321.896;266.452;0;274.018;,0.989844;0.989877;0.966991;0.995389;0;0.986519;,38824,0,-1,0,0
3,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_005-Pedr...,Q86U42,Q86U42,NaN,PABPN1,1100.290,1097.770,1054.950,1100.290,...,1.64526,0.048166,291.943;290.775;336.115;278.801;335.203;275.14;,291.943;290.775;336.115;278.223;323.855;275.14;,0.981929;0.990157;0.98916;0.980787;0.837441;0....,38920,0,-1,0,0
4,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_006-Pedr...,Q86U42,Q86U42,NaN,PABPN1,922.174,872.184,974.651,922.174,...,1.11004,0.095860,341.126;289.354;368.734;287.993;345.485;293.055;,341.126;289.354;367.433;287.993;345.485;293.055;,0.995846;0.955882;0.954787;0.98977;0.956884;0....,38920,0,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337304,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_003-Pedr...,Q8N183,Q8N183,NaN,NDUFAF2,246.667,259.171,303.202,246.667,...,1.42733,0.181841,232.362;201.957;71.3371;28.9291;50.4414;31.6606;,160.288;151.825;57.45;28.9291;34.87;23.2895;,0.887156;0.832145;0.968317;0.986023;0.907671;0...,32518,0,-1,0,0
337305,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_004-Pedr...,Q8N183,Q8N183,NaN,NDUFAF2,363.494,336.399,321.211,363.494,...,2.29118,0.070047,243.367;192.129;78.467;41.6598;56.605;34.7141;,243.367;192.129;78.467;41.6598;54.4399;34.6932;,0.992729;0.990228;0.993156;0.950967;0.90402;0....,32550,0,-1,0,0
337306,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_005-Pedr...,Q8N183,Q8N183,NaN,NDUFAF2,286.838,298.083,300.809,286.838,...,0.85119,0.179468,184.248;171.89;73.4033;29.1862;50.7739;32.5241;,184.248;171.89;73.4033;29.1862;46.6173;32.1469;,0.948194;0.993315;0.995006;0.864053;0.904326;0...,32614,0,-1,0,0
337307,X:\data\PXD002952\HYE124_TTOF6600_32fix_lgille...,HYE124_TTOF6600_32fix_lgillet_I150211_006-Pedr...,Q8N183,Q8N183,NaN,NDUFAF2,369.213,333.784,332.248,369.213,...,1.31142,0.000373,259.534;198.223;79.837;39.1171;56.5479;33.2562;,250.259;198.223;79.837;39.1171;48.455;32.3012;,0.967744;0.983911;0.977094;0.946098;0.894563;0...,32678,0,-1,0,0


# OSW

In [11]:


import pandas as pd
import numpy as np

from scipy import stats

import os 

os.chdir("/home/ptruong/git/dia_sum/scripts/PXD002952")

from q_value import qvalues
from triqler_output_to_df import  parse_triqler
os.chdir("/hdd_14T/data/PXD002952/osw_res_20210303/hye124/ttof6600/32fix/full_ts_v")



# filename has different formatting, we need to change number or implement regex.
experiment_id_mapper = lambda x: x.split("_")[5]
sample_id_mapper = lambda x: x.split("_")[8] #hye124 
specie_mapper = lambda x: x.split("_")[-1]
def decoy_mapper(x):
    if x.split("_")[0] == "DECOY":
        return True
    else:
        return False
    
def read_in_and_filter(filename, m_score_treshold = 0.01):  
    print(filename)
    df = pd.read_csv(filename, sep = "\t")
    df = df[df.decoy != 1]
    df = df[df.m_score < m_score_treshold] # filter away crap, so all values should be good... we take average of top3 here
    print(str(len(df)) + " significantly identified peptides at " + str(m_score_treshold) + " FDR-treshold.")
    print("")
    df["experiment_id"] = df["filename"].map(experiment_id_mapper)
    df["sample_id"] = df["filename"].map(sample_id_mapper)
    sample_id = df.sample_id[0]
    experiment_id = df.experiment_id[0]     
    def top3(df):
        df = (df.groupby('ProteinName')['Intensity'].apply(lambda x: x.nlargest(3).mean() if len(x.nlargest(3)) >= 2 else np.nan)
                  .reset_index())
        #print(df.isna().sum())
        return df
    df_reduced = df[["ProteinName", "Intensity"]]
    df_protein = top3(df_reduced)
    df = df_protein
    df["specie"] = df.ProteinName.map(specie_mapper)
    midx = pd.MultiIndex(levels = [[sample_id],[experiment_id]], codes = [[0],[0]], names = ["sample_id", "experiment_id"])
    df = df.set_index(["specie", "ProteinName"])
    df = pd.DataFrame(df.values, columns = midx, index = df.index)
    
    return df

In [12]:
print(filename)
df = pd.read_csv(filename, sep = "\t")
df = df[df.decoy != 1]
df = df[df.m_score < m_score_treshold]

NameError: name 'filename' is not defined

In [13]:
df = pd.read_csv("diaNN.tsv", sep = "\t")


FileNotFoundError: [Errno 2] No such file or directory: 'diaNN.tsv'